Quebrada Sonadora Nutrient Exports
---
Fluxes were calculated using the “loadflex” program (Appling et al. 2015), which is an enhancement of the widely used USGS “LOADEST” model (Runkel et al. 2004) as implemented in R. Both loadflex and LOADEST rely primarily on relationships between concentration of a solute and instantaneous discharge at the time of sampling to estimate concentrations when measured values of river chemistry are not available. In practice, and in our case, this often means that the 15-minute record of discharge at a given station is paired with an estimate of concentration based on weekly grab samples that span a range of discharge conditions.  

First load some r packages we will use.

In [1]:
library('loadflex')
library('rloadest')

Loading required package: smwrBase
Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: smwrGraphs
USGS Support Package: https://owi.usgs.gov/R/packages.html#support
Loading required package: smwrStats
This information is preliminary or provisional and
is subject to revision. It is being provided to meet
the need for timely best science. The information
has not received final approval by the U.S. Geological
Survey (USGS) and is provided on the condition that
neither the USGS nor the U.S. Government shall be held
liable for any damages resulting from the authorized
or unauthorized use of the information.

****Orphaned Package****
This package is looking for a new maintainer. For more information, 
see: https://owi.usgs.gov/R/packages.html#orphan
Loading required package: smwrQW
Loading required package: dataRetrieval
This information is preliminary or provisional and
is subject to re

Initially here we will load discharge and weekly grab sample data from csv files. This notebook details nutrient fluxes for the Quebrada Sonadora sampling site which has a watershed area of 261.58 hectares. 

In [2]:
QS_chem <- readRDS("data/QS_chem_UNH.rds")
QSDischargeShort <- readRDS("data/QS_Discharge_USGS.rds")
QSWatershedArea <- 261.5888


Calculate flux totals and mean annual concentration for Na. 

In [3]:
QS_chemNa <- QS_chem[complete.cases(QS_chem["Na_mg_L"]),]
QS_chemNa <- QS_chemNa[complete.cases(QS_chemNa["CFS"]),]


meta <- metadata(constituent="Na_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1")


lr <- loadReg2(loadReg(Na_mg_L ~ model(1), data=QS_chemNa,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))


lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=QS_chemNa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

# preds_lrNa <- predictSolute(lc, "flux",interval="prediction", QSDischargeShort, se.pred=TRUE, date=TRUE)

preds_lcNa <- predictSolute(lc, "flux",interval="prediction", QSDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNa <- aggregateSolute(preds_lcNa,meta, format="flux total", se.preds=preds_lcNa$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=QS_chemNa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcNaConc <- predictSolute(lcConc, "conc",interval="prediction", QSDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcNaConc <- aggregateSolute(preds_lcNaConc,meta, format="conc", se.preds=preds_lcNaConc$se.pred, agg.by="calendar year")

aggs_lcNa$Na_Kg_Ha_yr <- aggs_lcNa$Flux_Total / QSWatershedArea
aggs_lcNa$Na_mg_l <- aggs_lcNaConc$Conc
FluxTtotalsdf <- aggs_lcNa[, c('Na_mg_l','Na_Kg_Ha_yr','Calendar_Year')]

You are fitting an rloadest model (loadReg). Please remember to cite both citation('loadflex') and citation('rloadest').
Warning message in loadReg(Na_mg_L ~ model(1), data = QS_chemNa, flow = "CFS", dates = "date", :
"The minimum spacing between observed loads is 6 days. The time between observations should be at least  7 days to avoid autocorrelation issues."

Calculate flux totals and mean annual concentration for Ca. 

In [7]:
QS_chemCa <- QS_chem[complete.cases(QS_chem["Ca_mg_L"]),]
QS_chemCa <- QS_chemCa[complete.cases(QS_chemCa["CFS"]),]

meta <- metadata(constituent="Ca_mg_L", flow="CFS", 
                 dates="date", conc.units="mg/L", flow.units="cfs", load.units="kg", 
                 load.rate.units="kg d^-1", station="Rio Piedras, PR")


lr <- loadReg2(loadReg(Ca_mg_L ~ model(1), data=QS_chemCa,
                       flow="CFS", dates="date", time.step="instantaneous", 
                       flow.units="cfs", conc.units="mg/L", load.units="kg"))

lc <- loadComp(reg.model=lr, interp.format="flux", 
               interp.data=QS_chemCa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lrCa <- predictSolute(lc, "flux",interval="prediction", QSDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lrCa <- aggregateSolute(preds_lrCa,meta, format="flux total", se.preds=preds_lrCa$se.pred, agg.by="calendar year")

lcConc <- loadComp(reg.model=lr, interp.format="conc", 
                   interp.data=QS_chemCa, abs.or.rel.resids="relative", interp.function=rectangularInterpolation)

preds_lcCaConc <- predictSolute(lcConc, "conc",interval="prediction", QSDischargeShort, se.pred=TRUE, date=TRUE)

aggs_lcCaConc <- aggregateSolute(preds_lcCaConc,meta, format="conc", se.preds=preds_lcCaConc$se.pred, agg.by="calendar year")

aggs_lrCa$Ca_Kg_Ha_yr <- aggs_lrCa$Flux_Total / QSWatershedArea
aggs_lrCa$Ca_mg_l <- aggs_lcCaConc$Conc
FluxTtotalsdfCa <- aggs_lrCa[, c('Ca_mg_l','Ca_Kg_Ha_yr')]
FluxTtotalsdfCa <- merge(FluxTtotalsdfCa,FluxTtotalsdf)
FluxTtotalsdf

Warning message in loadReg(Ca_mg_L ~ model(1), data = QS_chemCa, flow = "CFS", dates = "date", :
"The minimum spacing between observed loads is 6 days. The time between observations should be at least  7 days to avoid autocorrelation issues."

Ca_mg_l,Ca_Kg_Ha_yr,Calendar_Year
2.703553,56.91087,2009
2.612966,108.80531,2010
2.826335,129.89182,2011
2.672091,103.35769,2012
2.518787,93.10251,2013
2.920442,75.58819,2014


In [6]:
FluxTtotalsdf

Ca_mg_l,Ca_Kg_Ha_yr,Calendar_Year
2.703553,56.91087,2009
2.612966,108.80531,2010
2.826335,129.89182,2011
2.672091,103.35769,2012
2.518787,93.10251,2013
2.920442,75.58819,2014
